In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import statsmodels

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'
df = pd.read_excel(data_path)

In [ ]:
df = df[df.Comment != 'Reanimation']

In [ ]:
# encode eineteilung
df['Einteilung_encoded'] = df['Einteilung (Hauptdiagnose)'].map({'Krankheit': 0, 'Unfall': 1})
df['Sverweis_Geschlecht_encoded'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})
df.loc[(df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (
            df['Kategorie (Hauptdiagnose)'] == 'Respiratorisch'), 'Kategorie (Hauptdiagnose)'] = 'Trauma_Respiratorisch'
# numerical encoding for Eineteilung and Kategorie
df['Kategorie_encoded'] = df['Kategorie (Hauptdiagnose)'].map({'Zentralnervensystem': 0,
                                                                         'Krampfanfall': 1,
                                                                         'Herz-Kreislauf': 2,
                                                                         'Intoxikation': 3,
                                                                         'Respiratorisch': 4,
                                                                         'SHT': 5,
                                                                         'Polytrauma': 6,
                                                                         'Verbrennung': 7,
                                                                         'Trauma_Respiratorisch': 8})

In [ ]:
# categorise age in >= 65 and < 65
df['Age_categorised'] = df['Alter  (Patient)'].apply(lambda x: 1 if x >= 65 else 0)

## Continuous

Univariate analysis

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Alter  (Patient)']], distr='probit')
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

adjusted

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Einteilung_encoded', 'Alter  (Patient)', 'Sverweis_Geschlecht_encoded']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']], distr='probit')
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

## Categorical

In [ ]:
# univariate analysis
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Age_categorised']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Age_categorised']], distr='probit')
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

In [ ]:
temp_df = df[['GCS (Erstbefund)', 'Einteilung_encoded', 'Age_categorised', 'Sverweis_Geschlecht_encoded']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Age_categorised']], distr='probit')
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

Plot    

In [ ]:
# gcs vs age (continuous)
sns.set_theme(style='whitegrid')
sns.set_context('talk')
sns.set_palette('Set2')
sns.set_style('whitegrid')
plt.figure(figsize=(10, 7))
sns.scatterplot(data=df, x='Alter  (Patient)', y='GCS (Erstbefund)', hue='Einteilung (Hauptdiagnose)')
plt.show()


In [ ]:
# gcs vs age (categorical)
sns.set_theme(style='whitegrid')
sns.set_context('talk')
sns.set_palette('Set2')
sns.set_style('whitegrid')
plt.figure(figsize=(10, 7))
sns.boxplot(data=df, x='Age_categorised', y='GCS (Erstbefund)')
plt.show()